In [2]:
import gym

# Random policy ----------------------------------
env = gym.make("Taxi-v2").env
for i in range(10):   
    env.reset()
    epochs = 0
    penalties, reward = 0,0
    frames = []
    done = False

    while not done:
        # action_space.sample() returns a random action
        action = env.action_space.sample()
        # play the random action
        state, reward, done, info = env.step(action)    
        if reward == -10:
            penalties += 1        
        frames.append({
            'frame': env.render(mode='ansi'),
            'state': state,
            'action': action,
            'reward': reward
            }
        )    
        epochs += 1
    print("Timesteps taken: {} with penalties: {}".format(epochs, penalties))



Timesteps taken: 3068 with penalties: 1002
Timesteps taken: 891 with penalties: 277
Timesteps taken: 514 with penalties: 174
Timesteps taken: 6100 with penalties: 2034
Timesteps taken: 3539 with penalties: 1175
Timesteps taken: 2645 with penalties: 896
Timesteps taken: 2572 with penalties: 855
Timesteps taken: 314 with penalties: 113
Timesteps taken: 1849 with penalties: 573
Timesteps taken: 915 with penalties: 316


In [3]:
%%time
import gym
import time
import numpy as np
import random
from IPython.display import clear_output


env = gym.make("Taxi-v2").env
env.reset()

# Q Learning ---------------------------

# 500 x 6 table for storing Q values
q_table = np.zeros([env.observation_space.n, env.action_space.n])

# Hyperparameters
alpha = 0.1
gamma = 0.6
epsilon = 0.1

# For plotting metrics
all_epochs = []
all_penalties = []

# Train agent for 100000 episodes, currently takes 1 minute to complete
for i in range(1, 100001):
    state = env.reset()
    epochs, penalties, reward, = 0, 0, 0
    done = False
    
    while not done:
        #Assume that with low epsilon a random action will be selected less often
        if random.uniform(0, 1) < epsilon:
            action = env.action_space.sample() # Explore action space
        else:
            # Select the highest weighted action from the Q Table
            action = np.argmax(q_table[state]) # Exploit learned values

        next_state, reward, done, info = env.step(action)         
        old_value = q_table[state, action]
        next_max = np.max(q_table[next_state])   
        
        # Q Learning algorithm
        new_value = (1 - alpha) * old_value + alpha * (reward + gamma * next_max)
        
        q_table[state, action] = new_value
        if reward == -10:
            penalties += 1

        state = next_state
        epochs += 1
        
    if i % 100 == 0:
        clear_output(wait=True)
        print(f"Episode: {i}")

print("Training finished.\n")

Episode: 100000
Training finished.

Wall time: 1min 10s


In [5]:
"""Evaluate agent's performance after Q-learning"""

total_epochs, total_penalties, total_rewards = 0, 0, 0

episodes = 10

for _ in range(episodes):
    state = env.reset()
    epochs, penalties, reward = 0, 0, 0
    
    done = False
    
    while not done:
        action = np.argmax(q_table[state])
        state, reward, done, info = env.step(action)

        if reward == -10:
            penalties += 1

        epochs += 1

    total_penalties += penalties
    total_epochs += epochs
    total_rewards += reward
    print("Timesteps taken: {} with Penalties incurred: {}".format(epochs, penalties))

print(f"Results after {episodes} episodes:")
print(f"Average rewards per move: {total_rewards / total_epochs}")
print(f"Average timesteps per episode: {total_epochs / episodes}")
print(f"Average penalties per episode: {total_penalties / episodes}")



Timesteps taken: 14 with Penalties incurred: 0
Timesteps taken: 13 with Penalties incurred: 0
Timesteps taken: 11 with Penalties incurred: 0
Timesteps taken: 11 with Penalties incurred: 0
Timesteps taken: 12 with Penalties incurred: 0
Timesteps taken: 12 with Penalties incurred: 0
Timesteps taken: 13 with Penalties incurred: 0
Timesteps taken: 12 with Penalties incurred: 0
Timesteps taken: 15 with Penalties incurred: 0
Timesteps taken: 10 with Penalties incurred: 0
Results after 10 episodes:
Average rewards per move: 1.6260162601626016
Average timesteps per episode: 12.3
Average penalties per episode: 0.0


In [7]:
# This cell's contents are sourced from: 
# https://github.com/allanbreyes/gym-solutions/blob/master/analysis/mdp.py

#These functions are to perform Temporal Difference Learning of the Taxi problem

import gym
import matplotlib.pyplot as plt
import numpy as np
import os
import time

def timing(f):
    def wrap(*args):
        time1 = time.time()
        ret = f(*args)
        time2 = time.time()
        #print '%s function took %0.3f ms' % (f.func_name, (time2-time1)*1000.0)
        return ret
    return wrap

def policy_iteration(problem, R=None, T=None, gamma=0.9, max_iterations=10**6, delta=10**-3):
    """ Runs Policy Iteration on a gym problem """
    num_spaces = problem.observation_space.n
    num_actions = problem.action_space.n

    # Initialize with a random policy and initial value function
    policy = np.array([problem.action_space.sample() for _ in range(num_spaces)])
    value_fn = np.zeros(num_spaces)

    # Get transitions and rewards
    if R is None or T is None:
        R, T = evaluate_rewards_and_transitions(problem)

    # Iterate and improve policies
    for i in range(max_iterations):
        previous_policy = policy.copy()

        for j in range(max_iterations):
            previous_value_fn = value_fn.copy()
            Q = np.einsum('ijk,ijk -> ij', T, R + gamma * value_fn)
            value_fn = np.sum(encode_policy(policy, (num_spaces, num_actions)) * Q, 1)

            if np.max(np.abs(previous_value_fn - value_fn)) < delta:
                break

        Q = np.einsum('ijk,ijk -> ij', T, R + gamma * value_fn)
        policy = np.argmax(Q, axis=1)

        if np.array_equal(policy, previous_policy):
            break

    # Return optimal policy
    return policy, i + 1

def value_iteration(problem, R=None, T=None, gamma=0.9, max_iterations=10**6, delta=10**-3):
    """ Runs Value Iteration on a gym problem """
    value_fn = np.zeros(problem.observation_space.n)
    if R is None or T is None:
        R, T = evaluate_rewards_and_transitions(problem)

    for i in range(max_iterations):
        previous_value_fn = value_fn.copy()
        Q = np.einsum('ijk,ijk -> ij', T, R + gamma * value_fn)
        value_fn = np.max(Q, axis=1)

        if np.max(np.abs(value_fn - previous_value_fn)) < delta:
            break

    # Get and return optimal policy
    policy = np.argmax(Q, axis=1)
    return policy, i + 1

def evaluate_rewards_and_transitions(problem, mutate=False):
    # Enumerate state and action space sizes
    num_states = problem.observation_space.n
    num_actions = problem.action_space.n

    # Intiailize T and R matrices
    R = np.zeros((num_states, num_actions, num_states))
    T = np.zeros((num_states, num_actions, num_states))

    # Iterate over states, actions, and transitions
    for state in range(num_states):
        for action in range(num_actions):
            for transition in problem.env.P[state][action]:
                probability, next_state, reward, done = transition
                R[state, action, next_state] = reward
                T[state, action, next_state] = probability

            # Normalize T across state + action axes
            T[state, action, :] /= np.sum(T[state, action, :])

    # Conditionally mutate and return
    if mutate:
        problem.env.R = R
        problem.env.T = T
    return R, T

def encode_policy(policy, shape):
    """ One-hot encodes a policy """
    encoded_policy = np.zeros(shape)
    encoded_policy[np.arange(shape[0]), policy] = 1
    return encoded_policy

In [8]:
# TD Learning -----------------------------------------------
env = gym.make("Taxi-v2")

# Calling value_itertation() sets the values in the value_policy table.
# The value_policy table is used in the next step for path traversal
value_policy, iters = value_iteration(env)


In [95]:
"""Evaluate agent's performance after TD Learning"""

total_epochs, total_penalties, total_rewards = 0, 0, 0

episodes = 10

for _ in range(episodes):
    state = env.reset()
    epochs, penalties, reward = 0, 0, 0
    done = False
    
    while not done:
        # Get the action as dictated in the value_policy table
        action = value_policy[state]
        state, reward, done, info = env.step(action)
        
        if reward == -10:
            penalties += 1

        epochs += 1

    print("Timesteps taken: {} with Penalties incurred: {}".format(epochs, penalties))
    total_penalties += penalties
    total_epochs += epochs
    total_rewards += reward
    
print(f"Results after {episodes} episodes:")
print(f"Average rewards per move: {total_rewards / total_epochs}")
print(f"Average timesteps per episode: {total_epochs / episodes}")
print(f"Average penalties per episode: {total_penalties / episodes}")

Timesteps taken: 17 with Penalties incurred: 0
Timesteps taken: 11 with Penalties incurred: 0
Timesteps taken: 15 with Penalties incurred: 0
Timesteps taken: 13 with Penalties incurred: 0
Timesteps taken: 14 with Penalties incurred: 0
Timesteps taken: 12 with Penalties incurred: 0
Timesteps taken: 16 with Penalties incurred: 0
Timesteps taken: 13 with Penalties incurred: 0
Timesteps taken: 16 with Penalties incurred: 0
Timesteps taken: 14 with Penalties incurred: 0
Results after 10 episodes:
Average rewards per move: 1.4184397163120568
Average timesteps per episode: 14.1
Average penalties per episode: 0.0
